# part0: imports

In [1]:
import os, sys, pathlib
from pprint import pprint 
from importlib import reload
import logging
logging.basicConfig(level=logging.ERROR)
import warnings
warnings.simplefilter("ignore")


import pandas as pd
import numpy as np
import xarray as xr
from sklearn.decomposition import PCA
import scipy.linalg as linalg

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from matplotlib.ticker import MaxNLocator


from tools import utilityTools as utility
from tools import dataTools as dt
import pyaldata as pyal

%matplotlib inline

root = pathlib.Path("/data")

# Compare different epochs

the idea is to see whether canonical axes between 2 animals provide a higher VAF for time epochs in the trial that they have not been trained on, compared to, for example, M1-PMd axes in a single animal.


## VAF in a subspace

Check the VAF for a single session as a test


In [2]:
animal= 'Chewie'
fname = root / animal / "Chewie_CO_CS_2016-10-21.mat"

df = pyal.mat2dataframe(fname, shift_idx_fields=True)

preprocessing

In [3]:
def get_target_id(trial):
    return int(np.round((trial.target_direction + np.pi) / (0.25*np.pi))) - 1

df = pyal.select_trials(df, df.result== 'R')
df = pyal.select_trials(df, df.epoch=='BL')
df = pyal.remove_low_firing_neurons(df, "M1_spikes", 1)
df = pyal.remove_low_firing_neurons(df, "PMd_spikes", 1)

df = pyal.add_firing_rates(df, 'smooth')

df["target_id"] = df.apply(get_target_id, axis=1)  # add a field `target_id` with int values

/home/mostafa/Repositories/MyPyalData/pyaldata/tools.py:940: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")
/home/mostafa/Repositories/MyPyalData/pyaldata/tools.py:940: UserWarning: Assuming spikes are actually spikes and dividing by bin size.
  utils.warnings.warn("Assuming spikes are actually spikes and dividing by bin size.")


applying PCA for 300ms post movement onset

In [4]:
df_ = pyal.restrict_to_interval(df, start_point_name='idx_movement_on', rel_start=0, rel_end=30)

M1_rates = np.concatenate(df_.M1_rates.values, axis=0)
M1_rates -=np.mean(M1_rates,axis=0)

M1_model = PCA(n_components=10, svd_solver='full');
M1_model.fit(M1_rates);
df_ = pyal.apply_dim_reduce_model(df_, M1_model, 'M1_rates', 'M1_pca');


PMd_rates = np.concatenate(df_.PMd_rates.values, axis=0)
PMd_rates -=np.mean(PMd_rates,axis=0)

PMd_model = PCA(n_components=10, svd_solver='full');
PMd_model.fit(PMd_rates);
df_ = pyal.apply_dim_reduce_model(df_, PMd_model, 'PMd_rates', 'PMd_pca');

to make life easy, just limit everything to 1 target

In [5]:
df__= pyal.select_trials(df_, df_.target_id ==3)

CCA on m1-pmd

In [6]:
d0 = np.concatenate(df__['M1_pca'].values, axis=0)
d1 = np.concatenate(df__['PMd_pca'].values, axis=0)

n_samples = min ([d0.shape[0], d1.shape[0]])
d0 = d0[:n_samples,:]
d1 = d1[:n_samples,:]

A, B, r, _, _ = dt.canoncorr(d0, d1, fullReturn=True)

In [7]:
print(f'the CCs are:{r}')

the CCs are:[0.9246411  0.79977804 0.67285567 0.56713524 0.49359452 0.36927065
 0.34098189 0.25425819 0.13815213 0.04389754]


---

From Bence's code [here](https://github.com/BeNeuroLab/pysubspaces/blob/4b7c7512ccdff8fdbd1334cb6c3acc8f5bdbf3d6/pysubspaces/subspaces.py#L29).

In [8]:
def variance_in_subspace(X, W):
    """
    Variance in a given subspace
    Parameters
    ----------
    X : 2D np.ndarray
        n_samples x n_features data array
    W : 2D np.ndarray
        n_components x n_features projection matrix
    Returns
    -------
    variance in the subspace
    """
    return np.trace(W @ np.cov(X.T) @ W.T)

In [9]:
variance_in_subspace (d0, A.T), variance_in_subspace (d1, B.T)

(10.000000000000002, 9.999999999999996)

weirdly specific?! but what is the total variance? what does it mean?
- can I shuffle the weights in the prrojection matrix and repeat mulriple times to estimate a band of VAF by chance?
    - maybe, check the matlab code
- this is probably based on [this paper](https://www.nature.com/articles/ncomms13239#Sec9).

---

Based on the Gallego Nat Comm 2018 paper and some playing (data and projection matrices in the paper are row-wise): 
$$\%VAF=\frac{norm(X)-norm(X-XC^TA(A^TA)^{-1}A^TC)}{norm(X)}$$
where:
- $A$ is the CCA output, the output of the `canoncorr` function
- $C$ is the `PCA_model.components_`
- $X$ is the data matrix, $T\times n$ with $T$ time points and $n$ neurons, and each neuron is **zero mean**
- $norm$ is sum of squared elements  

In [10]:
C = M1_model.components_
X = M1_rates
norm = lambda m:np.sum(m**2)

VAF = norm(X) - norm(X-X@C.T@A@linalg.inv(A.T@A)@A.T@C)
VAF /= norm(X)

VAF

0.6482088310037387

Seems to give reasonable results: lowering number of PCs reduces the VAF.

However, there are problemss:
- the formulation in the paper, gives a VAF for each dimension, so the formula above should be applied to individual axes.
- It is already implemented in MATLAB [here](https://github.com/limblab/proc-juan/blob/65dd81a1fc0ddb23def8039df9053a2be3e9bf88/paper_cross-task_manifold_review/vaf_CCs.m#L114), which uses the VAF function from the dPCA paper [here](https://github.com/machenslab/dPCA/blob/master/matlab/dpca_explainedVariance.m). I should translate that to compare?!?


### dimension-wise VFA

In [11]:
D = linalg.inv(A.T@A)@A.T@C
E = C.T@A

VAFs=np.empty((C.shape[0],))

for comp in range(C.shape[0]):

    VAF = norm(X - X @ E[:,comp:comp+1] @ D[comp:comp+1,:]) / norm(X)
    VAFs[comp] = 1-VAF

print(f'{VAFs} \n sums up to: {sum(VAFs)}')

[ 0.01229023  0.10043615  0.06834629 -0.00385621  0.05905853  0.01711739
 -0.02952448 -0.17102626 -0.01291792  0.01157017] 
 sums up to: 0.05149389724350406
